**aaa**

In [1]:
import pandas as pd
from datetime import datetime

In [18]:
# Load Datasets
df_analytics = pd.read_csv("datasets/archivo_combinado_updated.csv")
df_images = pd.read_csv("datasets/imagedataFINAL27022025.csv")

In [19]:
df_images.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 274 entries, 0 to 273
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   nombre_archivo    274 non-null    object
 1   fecha             274 non-null    object
 2   objeto_principal  274 non-null    object
 3   color_dominante   274 non-null    object
 4   emocion_primaria  274 non-null    object
dtypes: object(5)
memory usage: 10.8+ KB


In [4]:
df_analytics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 702 entries, 0 to 701
Data columns (total 22 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Identificador de la publicación  702 non-null    int64  
 1   Identificador de la cuenta       702 non-null    int64  
 2   Nombre de usuario de la cuenta   702 non-null    object 
 3   Nombre de la cuenta              702 non-null    object 
 4   Descripción                      302 non-null    object 
 5   Duración (segundos)              702 non-null    int64  
 6   Hora de publicación              702 non-null    object 
 7   Enlace permanente                13 non-null     object 
 8   Tipo de publicación              702 non-null    object 
 9   Comentario sobre los datos       0 non-null      float64
 10  Fecha                            702 non-null    object 
 11  Alcance                          642 non-null    float64
 12  Me gusta              

In [5]:
 #Step 2. Prepare the dataset for analysis
df_interim = (
    df_analytics
    .copy()
    .set_axis(
        df_analytics.columns.str.replace(' ', '_') # reemplazar espacio por _
        .str.replace('á', 'a') # reemplazar
        .str.replace('é', 'e') # reemplazar
        .str.replace('í', 'i') # reemplazar
        .str.replace('ó', 'o') # reemplazar
        .str.replace('ú', 'u') # reemplazar
        .str.replace(r'\W','',regex=True) # reemplazar caracteres no alfabéticos por nada
        .str.lower() # el resultado lo pasamos a minúsculas
        .str.slice(0,40), axis=1, # slice sirve para cortar el texto a un máximo de 40 caracteres
    )
    

)
df_interim.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 702 entries, 0 to 701
Data columns (total 22 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   identificador_de_la_publicacion  702 non-null    int64  
 1   identificador_de_la_cuenta       702 non-null    int64  
 2   nombre_de_usuario_de_la_cuenta   702 non-null    object 
 3   nombre_de_la_cuenta              702 non-null    object 
 4   descripcion                      302 non-null    object 
 5   duracion_segundos                702 non-null    int64  
 6   hora_de_publicacion              702 non-null    object 
 7   enlace_permanente                13 non-null     object 
 8   tipo_de_publicacion              702 non-null    object 
 9   comentario_sobre_los_datos       0 non-null      float64
 10  fecha                            702 non-null    object 
 11  alcance                          642 non-null    float64
 12  me_gusta              

In [6]:
df_interim.drop("identificador_de_la_publicacion", axis=1, inplace=True)
df_interim.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 702 entries, 0 to 701
Data columns (total 21 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   identificador_de_la_cuenta      702 non-null    int64  
 1   nombre_de_usuario_de_la_cuenta  702 non-null    object 
 2   nombre_de_la_cuenta             702 non-null    object 
 3   descripcion                     302 non-null    object 
 4   duracion_segundos               702 non-null    int64  
 5   hora_de_publicacion             702 non-null    object 
 6   enlace_permanente               13 non-null     object 
 7   tipo_de_publicacion             702 non-null    object 
 8   comentario_sobre_los_datos      0 non-null      float64
 9   fecha                           702 non-null    object 
 10  alcance                         642 non-null    float64
 11  me_gusta                        702 non-null    int64  
 12  veces_que_se_compartio          702 

In [7]:
df_interim.drop("identificador_de_la_cuenta", axis=1, inplace=True)
df_interim.drop("nombre_de_usuario_de_la_cuenta", axis=1, inplace=True)
df_interim.drop("nombre_de_la_cuenta", axis=1, inplace=True)
df_interim.drop("descripcion", axis=1, inplace=True)
df_interim.drop("enlace_permanente", axis=1, inplace=True)
df_interim.drop("tipo_de_publicacion", axis=1, inplace=True)
df_interim.drop("comentario_sobre_los_datos", axis=1, inplace=True)
df_interim.drop("fecha", axis=1, inplace=True)

df_interim.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 702 entries, 0 to 701
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   duracion_segundos       702 non-null    int64  
 1   hora_de_publicacion     702 non-null    object 
 2   alcance                 642 non-null    float64
 3   me_gusta                702 non-null    int64  
 4   veces_que_se_compartio  702 non-null    int64  
 5   visitas_al_perfil       527 non-null    float64
 6   respuestas              702 non-null    int64  
 7   navegacion              637 non-null    float64
 8   clics_en_el_enlace      25 non-null     float64
 9   seguimientos            24 non-null     float64
 10  toques_en_stickers      24 non-null     float64
 11  visualizaciones         123 non-null    float64
 12  sentimiento             702 non-null    float64
dtypes: float64(8), int64(4), object(1)
memory usage: 71.4+ KB


In [8]:
# 2.2 Create the final dataset with the target in front, show result
df = (
    df_interim
    .copy()
    .reindex(
        columns=(
            ['hora_de_publicacion'] +
            [c for c in df_interim.columns.to_list() if c not in ['hora_de_publicacion']]
        )
    )
)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 702 entries, 0 to 701
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   hora_de_publicacion     702 non-null    object 
 1   duracion_segundos       702 non-null    int64  
 2   alcance                 642 non-null    float64
 3   me_gusta                702 non-null    int64  
 4   veces_que_se_compartio  702 non-null    int64  
 5   visitas_al_perfil       527 non-null    float64
 6   respuestas              702 non-null    int64  
 7   navegacion              637 non-null    float64
 8   clics_en_el_enlace      25 non-null     float64
 9   seguimientos            24 non-null     float64
 10  toques_en_stickers      24 non-null     float64
 11  visualizaciones         123 non-null    float64
 12  sentimiento             702 non-null    float64
dtypes: float64(8), int64(4), object(1)
memory usage: 71.4+ KB


In [9]:
# Renombrar la columna 'sentimiento' a 'sentimiento_score'
df.rename(columns={'sentimiento': 'sentimiento_score'}, inplace=True)

# Mostrar el DataFrame con la columna renombrada
print(df.head())

  hora_de_publicacion  duracion_segundos  alcance  me_gusta  \
0    10/01/2023 10:04                  0   1989.0         7   
1    12/31/2023 12:11                  5    527.0         6   
2    12/31/2023 10:17                  0    750.0         4   
3    12/29/2023 13:44                  0    555.0         8   
4    12/29/2023 10:09                  0    841.0         3   

   veces_que_se_compartio  visitas_al_perfil  respuestas  navegacion  \
0                       0                9.0           0      2019.0   
1                       0                1.0           1       514.0   
2                       0                3.0           0       762.0   
3                       0                NaN           2       620.0   
4                       0                3.0           0       840.0   

   clics_en_el_enlace  seguimientos  toques_en_stickers  visualizaciones  \
0                 NaN           NaN                 NaN              NaN   
1                 NaN           NaN 

In [10]:
# Eliminar las filas en las que la columna 'duracion_segundos' sea mayor a cero porque son videos y no los tengo en el dataset de imagenes
df = df[df['duracion_segundos'] <= 0]

# Eliminar la columna 'duracion_segundos'
df.drop('duracion_segundos', axis=1, inplace=True)
df.info()



<class 'pandas.core.frame.DataFrame'>
Index: 385 entries, 0 to 701
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   hora_de_publicacion     385 non-null    object 
 1   alcance                 359 non-null    float64
 2   me_gusta                385 non-null    int64  
 3   veces_que_se_compartio  385 non-null    int64  
 4   visitas_al_perfil       302 non-null    float64
 5   respuestas              385 non-null    int64  
 6   navegacion              357 non-null    float64
 7   clics_en_el_enlace      10 non-null     float64
 8   seguimientos            10 non-null     float64
 9   toques_en_stickers      16 non-null     float64
 10  visualizaciones         91 non-null     float64
 11  sentimiento_score       385 non-null    float64
dtypes: float64(8), int64(3), object(1)
memory usage: 39.1+ KB


In [11]:
# Mostrar el DataFrame resultante
df.head()

,hora_de_publicacion,alcance,me_gusta,veces_que_se_compartio,visitas_al_perfil,respuestas,navegacion,clics_en_el_enlace,seguimientos,toques_en_stickers,visualizaciones,sentimiento_score
0,10/01/2023 10:04,1989.0,7,0,9.0,0,2019.0,NaN,NaN,NaN,NaN,0.0
2,12/31/2023 10:17,750.0,4,0,3.0,0,762.0,NaN,NaN,NaN,NaN,0.0
3,12/29/2023 13:44,555.0,8,0,NaN,2,620.0,76.0,NaN,NaN,NaN,-2.0
4,12/29/2023 10:09,841.0,3,0,3.0,0,840.0,NaN,NaN,NaN,NaN,0.0
11,12/28/2023 07:21,401.0,1,0,4.0,0,389.0,NaN,NaN,NaN,NaN,-2.0


In [12]:
df_images.head()

,nombre_archivo,fecha,objeto_principal,color_dominante,emocion_primaria
0,IMG_20231222_182937_798.jpg,2023-12-22,potter's_wheel,#4e4f66,happy
1,IMG_20231106_195856_513.webp,2023-11-06,swing,#42454e,happy
2,IMG_20231103_173639_881.webp,2023-11-03,corn,#656b7c,happy
3,IMG_20231102_210840_705.webp,2023-11-02,altar,#686977,fear
4,IMG_20230417_194119_473.jpg,2023-04-17,jersey,#c3718a,sad


In [13]:
# Crear una función para convertir fechas y manejar formatos mixtos
def convertir_fecha(fecha):
    for fmt in ('%d/%m/%Y %H:%M', '%m/%d/%Y %H:%M'):
        try:
            return pd.to_datetime(fecha, format=fmt)
        except ValueError:
            continue
    return pd.NaT

# Aplicar la función de conversión a cada valor en la columna 'Hora de publicación'
df['hora_de_publicacion'] = df['hora_de_publicacion'].apply(convertir_fecha)

# Crear una columna 'fecha' basada en 'Hora de publicación'
df['fecha'] = df['hora_de_publicacion'].dt.date

# Asegurarse de que la columna 'fecha' sea del tipo datetime64[ns]
df['fecha'] = pd.to_datetime(df['fecha'], errors='coerce')

# Verificar los tipos de las columnas
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 385 entries, 0 to 701
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   hora_de_publicacion     385 non-null    datetime64[ns]
 1   alcance                 359 non-null    float64       
 2   me_gusta                385 non-null    int64         
 3   veces_que_se_compartio  385 non-null    int64         
 4   visitas_al_perfil       302 non-null    float64       
 5   respuestas              385 non-null    int64         
 6   navegacion              357 non-null    float64       
 7   clics_en_el_enlace      10 non-null     float64       
 8   seguimientos            10 non-null     float64       
 9   toques_en_stickers      16 non-null     float64       
 10  visualizaciones         91 non-null     float64       
 11  sentimiento_score       385 non-null    float64       
 12  fecha                   385 non-null    datetime64[ns]


In [14]:
# Asegurarse de que la columna 'fecha' sea del tipo datetime64[ns]
df_images['fecha'] = pd.to_datetime(df_images['fecha'], errors='coerce')
df_images.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 211 entries, 0 to 210
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   nombre_archivo    211 non-null    object        
 1   fecha             211 non-null    datetime64[ns]
 2   objeto_principal  211 non-null    object        
 3   color_dominante   211 non-null    object        
 4   emocion_primaria  211 non-null    object        
dtypes: datetime64[ns](1), object(4)
memory usage: 8.4+ KB


In [90]:
# Combinar los DataFrames basados en la columna 'fecha' y eliminar registros sin coincidencias
df_combinado = df.merge(df_images, on='fecha', how='inner')

# Mostrar el DataFrame combinado
print(df_combinado.head())


  hora_de_publicacion  alcance  me_gusta  veces_que_se_compartio  \
0 2023-12-31 10:17:00    750.0         4                       0   
1 2023-12-29 13:44:00    555.0         8                       0   
2 2023-12-29 13:44:00    555.0         8                       0   
3 2023-12-29 10:09:00    841.0         3                       0   
4 2023-12-29 10:09:00    841.0         3                       0   

   visitas_al_perfil  respuestas  navegacion  clics_en_el_enlace  \
0                3.0           0       762.0                 NaN   
1                NaN           2       620.0                76.0   
2                NaN           2       620.0                76.0   
3                3.0           0       840.0                 NaN   
4                3.0           0       840.0                 NaN   

   seguimientos  toques_en_stickers  visualizaciones  sentimiento_score  \
0           NaN                 NaN              NaN                0.0   
1           NaN                 

In [ ]:
# Descubrí que algunas entradas se duplicaron, entonces voy a borrarlas

# Guardar el DataFrame combinado en un nuevo archivo CSV
df_combinado.to_csv('final_datset.csv', index=False)